In [14]:
import numpy as np
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from xgboost import plot_importance

from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv("cleaned_data.csv", index_col=0)

df = df[df["CODE_GENDER"] != "XNA"] # filter out gender = XNA bc only 4 entries
categorical = ["NAME_CONTRACT_TYPE", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY"] # change categorical variables to 0, 1 encoding
for col in categorical:
  fst = df[col].iloc[0]
  df[col] = (df[col] == fst).astype(float)

df = df.drop(columns=["OWN_CAR_AGE"]) # drop this because we have the new categorical variables for this info

In [3]:
# drop entries with a nan
df = df.dropna(axis=0)

In [4]:
# figure out which columns only have one value
for i in df.columns:
    if len(pd.unique(df[i])) <= 1:
        print(i, pd.unique(df[i]))

FLAG_MOBIL [1]
Unknown [0]


In [5]:
df = df.drop(columns=["FLAG_MOBIL", "Unknown"]) #these columns now only have one value so they are useless for predicting --> remove them

In [6]:
# logistic regression
Xtrain = df.drop(columns=["TARGET", "SK_ID_CURR"], axis=1)
ytrain = df["TARGET"]

log_reg = sm.Logit(ytrain, Xtrain.astype(float)).fit()

print(log_reg.summary())

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1863: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix

In [7]:
# linear model 
x = sm.add_constant(Xtrain, prepend=False)
mod = sm.OLS(ytrain, x.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 TARGET   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     54.32
Date:                Thu, 27 Apr 2023   Prob (F-statistic):               0.00
Time:                        23:34:08   Log-Likelihood:                -22180.
No. Observations:              264894   AIC:                         4.466e+04
Df Residuals:                  264742   BIC:                         4.626e+04
Df Model:                         151                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
NAME_CONTRACT_

In [23]:
X = Xtrain
y = ytrain
seed = 100
test_size = 0.25

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [24]:
y_pred = model.predict(X_test)

In [25]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 92.36%


In [26]:
feat_dict= {}
for col, val in sorted(zip(X_train.columns, model.feature_importances_),key=lambda x:x[1],reverse=True):
  feat_dict[col]=val
feat_df = pd.DataFrame({'Feature':feat_dict.keys(),'Importance':feat_dict.values()})
feat_df

,Feature,Importance
0,Higher education,0.046046
1,Secondary / secondary special,0.024406
2,Working,0.024198
3,REGION_RATING_CLIENT_W_CITY,0.020610
4,car_owned_less_10,0.020432
...,...,...
154,Trade: type 1,0.000000
155,Trade: type 4,0.000000
156,Trade: type 5,0.000000
157,Transport: type 1,0.000000


In [27]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     61171
           1       0.42      0.00      0.01      5053

    accuracy                           0.92     66224
   macro avg       0.67      0.50      0.48     66224
weighted avg       0.89      0.92      0.89     66224



In [28]:
f1 = f1_score(y_test, y_pred)
f1

0.007841599686336012

In [29]:
y_pred_adjusted = []
for pred in model.predict_proba(X_test):
    if pred[1] >= 0.1:
        y_pred_adjusted.append(1)
    else:
        y_pred_adjusted.append(0)
print(classification_report(y_test, y_pred_adjusted))
f1_2 = f1_score(y_test, y_pred_adjusted)
print(f1_2)

              precision    recall  f1-score   support

           0       0.95      0.78      0.86     61171
           1       0.15      0.47      0.23      5053

    accuracy                           0.76     66224
   macro avg       0.55      0.63      0.54     66224
weighted avg       0.89      0.76      0.81     66224

0.2298211116972807
